# Hypercube - Preparación de ficheros de datos para la NN

Procesos para preparar los datos extraidos de Matlab con el script CreateMaskForMaterials.m y generar los ficheros de entrenamiento y test para la red neuronal.

El directorio donde se están ubicando todos los .csv que se extraen de los hypercubos capturados con la cámara es C:\\_DATA\\JOVISA


In [1]:
import pandas as pd
import glob
import os
import numpy as np
import math
import random
from FileGenerator import BatchGenerator
from FileGenerator import ValidationGenerator
from FileGenerator import SpectrumsCounter
from FileGenerator import FilesNumLines
from FileGenerator import FileNumLines
from neural_network_Estimator_v3 import RegressionHyperModel_1Layer


import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow import keras, optimizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout
import keras_tuner as kt
from keras_tuner import HyperModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

## Definición de funciones de utilidad en FileGenerator.py

### BatchGenerator
En FileGenerator.py tenemos el BatchGenerator que nos proporciona los batches de espectros desde los ficheros. Lo configuramos para que cada vez que se usa next sobre el generador, se devuelva un único espectro, tanto X como Y.

    Nota: En la versión original usada para las termicas teníamos uno para train y otro para test, pero como se le pasa el FileList que componen cada uno de los conjuntos el funcionamiento es el mismo. Aquí sólo hemos dejado el BatchGenerator como genérico.

Tenemos también varias funcione de utilidad

### Función NumLines
Creamos una función para calcular el número de líneas de un fichero.
La usaremos para definir las líneas por Chunk que utilizarmos para partir los ficheros en los mismos trozos

### SpectrumsCounter
Dado un conjutno de ficheros nos dice el número de líneas (espectros) en conjunto que tiene. (hay que ver si con NumLines va más rápido)

## Establecimiento de Path y lista de ficheros a procesar
Definimios el path de donde cogemos los ficheros .csv generados por Matlab, donde los espectros ya están preprocesados.
Este path es diferente en funciòn del ordenador donde estemos ejecutando, según los comentarios de la siguiente celda.

In [2]:
#Path de los CSV preprocesados
#Descomentar el que proceda.

#PATHS EN HCUBESERVER
#path = r'C:\_DATA\JOVISA\CSV'
#pathSplit = r'C:\_DATA\JOVISA\CSV\SPLIT' #
#trainPath = r'C:\_DATA\JOVISA\CSV\SPLIT\TRAIN'
#testPath = r'C:\_DATA\JOVISA\CSV\SPLIT\TEST'

#PATHS CINTA VERDE
path = r'.'
pathSplit = r'.\SPLIT'
trainPath = r'.\TRAIN'
testPath = r'.\TEST'
#PATHS DATOS CINTA NEGRA
#path = r'D:\_DATA\JOVISA_CSV_FOLDER\CSV\raw'
#pathSplit = r'D:\_DATA\JOVISA_CSV_FOLDER\CSV\SPLIT'
#trainPath = r'D:\_DATA\JOVISA_CSV_FOLDER\CSV\SPLIT\TRAIN'
#testPath = r'D:\_DATA\JOVISA_CSV_FOLDER\CSV\SPLIT\TEST'
#PATHS EN PORTATIL
#path = r'C:\_DATA\JOVISA\CSV'
#pathSplit = r'C:\_DATA\JOVISA\CSV\SPLIT' #
#trainPath = r'C:\_DATA\JOVISA\CSV\SPLIT\TRAIN'
#testPath = r'C:\_DATA\JOVISA\CSV\SPLIT\TEST'

#all_files = glob.glob(os.path.join(path , "tap-pet_roi_from_botellas2a80pct_0_preprocesed.csv"))
all_files = glob.glob(os.path.join(path , "*.csv"))
#print(all_files)

### Generación de ficheros con muestras de materiales

Los ficheros que matlab genera con los espectros tiene un tipo de material por fichero.
Hay que generar una secuencia aleatoria de espectros, de forma que la NN tenga muestras aleatoriamente generadas para el conjunto de entrenamiento y de test.

Como primer generamos un conjunto de NFiles ficheros cuyo contenido se obtiene de los ficheros .csv preprocesados de todos los materiales, añadiendo proporcionalmente (al numero de filas del fichero) el número de filas de cada uno de los materiales.

Para saber el número de lineas de un fichero usamos la función NumLines que hemos definido antes.
Vamos a generar nFiles=100 ficheros en el directorio .\SPLIT.
Cada fichero tendrá un trozo de cada uno de los ficheros .csv, para que haya muestras de cada material en todos los ficheros generados.
Para saber de un fichero cuantas lineas hay que coger para generar 100 trozos (chunks) usamos: math.ceil(nLines/nFiles).


In [ ]:
#TROCEA LOS FICHEROS PREPROCESADOS EN nFiles TROZOS, GRABANDO los nuevos ficheros con parte de cada uno de los originales
nFiles=100
for filename in all_files:
    cCounter=0
    nLines= FileNumLines(filename)
    linesChunks=math.ceil(nLines/nFiles)
    print("Procesando "+filename)
    with pd.read_csv(filename, sep=";", index_col=None, header=None, chunksize=linesChunks) as reader:
        for chunk in reader:
            oFile=pathSplit+"\\f"+str(cCounter)+".csv"
            #print(oFile)
            #print(chunk.size)
            chunk.to_csv(oFile, mode='a',sep=";", header=False, index=False)
            cCounter+=1
        #print(cCounter)

### Reordenación aleatoria de los espectros en los ficheros
Una vez que tenemos los ficheros en el directorio SPLIT, estos tienen un bloque de lineas consecutivas de espectros para cada tipo.
Lo que hay que hacer es modificar el orden de las líneas para que sea aleatorio.
Se modifica el orden aleatorio sobreescribiendo el fichero en el directorio de SPLIT.

In [ ]:
#de los ficheros trozeados anteriormente, los carga, reordena aleatoriamente y vuelve a grabarlos (sobreescribe)
splitFiles= glob.glob(os.path.join(pathSplit , "*.csv"))
#print(splitFiles)
for filename in splitFiles:
    df= pd.read_csv(filename, sep=";", index_col=None, header=None)
    df = df.sample(frac=1).reset_index(drop=True)
    df.to_csv(filename, index=False,sep=";", header=None)

# Generación de Training y Test Files

1. Crear directorios de train y test
2. Ordenar aleatoriamente los nFiles
3. Determinar cuantos van a train y a test y copiarlos a su dir
4. Definir el BatchGenerator y ValidationGenerator. Modificar FileGenerator.py para ello.



1. y 2. Creados y ordenados

In [ ]:
splitFiles= glob.glob(os.path.join(pathSplit , "*.csv"))
random.shuffle(splitFiles)
print(splitFiles)
#Definimos los porcentajes de test y train
pctTrain=.8

trainFiles=splitFiles[0:int(len(splitFiles)*pctTrain)]
testFiles=splitFiles[int(len(splitFiles)*pctTrain):len(splitFiles)]

#print(len(trainFiles))
#print(len(testFiles))




3. cuantos y moverlos a los directorios TRAIN y TEST

In [ ]:
for filename in trainFiles:
    basename = os.path.basename(filename)
    os.rename(filename, trainPath + "\\" + basename)

for filename in testFiles:
    basename = os.path.basename(filename)
    os.rename(filename, testPath + "\\" + basename)


### Definición de variables y Generador

- Cargamos las variables **trainFiles** y **testFiles** con el conjunto respectivo de ficheros para entrenamiento y ficheros para validación o test.
- Definimios un BatchGenerador temporal para capturar el número de Features


In [ ]:
#definimos las listas de ficheros
trainFiles= glob.glob(os.path.join(trainPath , "f*.csv"))
testFiles= glob.glob(os.path.join(testPath , "f*.csv"))
#print(trainFiles[0])
#Definimos el generador temporal con un único fichero para poder capturar luego los parámetros o features de la red.
#filelist=[trainFiles[0]]
filelist=trainFiles
#print(filelist)
batchsize=1000
traingenerator = BatchGenerator(filelist, batchsize, ';', validation=False)
print(traingenerator)


## Obtenemos el numero de parametros de entrada y el numero de tipos de materiales diferentes a partir del BatchGenerator

In [ ]:

#Obtenemos el número de parametros usando el generaodr definido con un único fichero.
#Al usar el generador nos da los dos valores y ya no los recuperamos, por lo que habrá que redefinir el generador más adelante.
x,y=next(traingenerator)
#print(x,y)
n_features=x.shape[1]
print(f"Número de Features: {n_features}")
n_clases=np.unique(y).shape[0]
print(n_clases)
#REINICIAMOS EL TRAINGENERATOR POR HABER PERDIDO LA PRIMERA EJECUCION
traingenerator = BatchGenerator(filelist, batchsize, ';', validation=False)

In [9]:
#definimos las listas de ficheros
trainFiles= glob.glob(os.path.join(trainPath , "f*.csv"))
testFiles= glob.glob(os.path.join(testPath , "f*.csv"))
batchsize=1000
traingenerator = BatchGenerator(trainFiles, batchsize, ';', validation=False)
testgenerator = BatchGenerator(testFiles, batchsize, ';', validation=False)


X_train,Y_train=next(traingenerator)
X_test,Y_test=next(testgenerator)

data_augmentation = False
only_size = None; # Puede ser [None, 4, 8, 16, 32, 64]

normalization = False
standarization = True

add_MDV = False
add_statistics = True
add_sizes = True
model_type=1
# if standarization or normalization:
#     if standarization:
#         scaler = StandardScaler()
#     else:
#         scaler = MinMaxScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)


input_shapex = (X_train.shape[1],)
print(input_shapex)

Files to process: 80
Separator ;


c:\Users\Fran Quiles\Documents\Tfg_IA\hipercubo\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Files to process: 20
Separator ;
(204,)


c:\Users\Fran Quiles\Documents\Tfg_IA\hipercubo\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
if model_type == 1:
    print(input_shapex)
    hypermodel_1 = RegressionHyperModel_1Layer(input_shape=input_shapex)
    print(hypermodel_1)
    tuner_hb1 = kt.Hyperband(
                    hypermodel_1,
                    objective=kt.Objective("val_recall", direction="max"),
                    max_epochs=1000,
                    factor=3,
                    directory='my_dir_model1',
                    project_name='Estimator_v3')
else:
    print("Option not available")
    exit(-1)
                    
                    
# Will stop training if the "val_loss" hasn't improved in 5 epochs.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_recall', patience=10)


if model_type == 1:
    tuner_hb1.search(X_train, Y_train, epochs=2000, validation_split=0.2, callbacks=[stop_early], verbose=1)
    tuner_hb1.results_summary()
    best_hps1=tuner_hb1.get_best_hyperparameters(num_trials=1)[0]


# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss',
                                  mode='min',
                                  patience=10,
                                  verbose=1,
                                  restore_best_weights=True)
]
if model_type == 1:
    model1 = tuner_hb1.hypermodel.build(best_hps1)
    history1 = model1.fit(X_train, Y_train, epochs=1000, callbacks=callbacks, validation_split=0.2)
    eval_result1 = model1.evaluate(X_test, Y_test)
    print("[test loss, test recall, test precision]:", eval_result1)

if model_type == 1:
    y_pred = model1.predict(X_test)

y_pred=np.argmax(y_pred, axis=1)
Y_test=np.argmax(Y_test, axis=1)

print("f1_score: %0.5f" % (f1_score(Y_test, y_pred, average="weighted")) )
print("precision_score: %0.5f" % (precision_score(Y_test, y_pred, average="weighted")) )
print("recall_score: %0.5f" % (recall_score(Y_test, y_pred, average="weighted")) )

cm = confusion_matrix(Y_test, y_pred)
#print(cm)
import matplotlib.pyplot as plt

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.array(['Plain','Edge','Texture']))
disp.plot()
plt.savefig('confusion_matrix_Layer' + str(model_type) + '.png')

(204,)


NameError: name 'input_shape' is not defined

- Obtenemos el numero total de espectro en el train set

In [ ]:
totalSpectrums = FilesNumLines(trainFiles)
print("Número total de espectros en todos los ficheros: ",totalSpectrums)
# totalSpectrums = SpectrumsCounter(trainFiles)
# print(totalSpectrums)


### DEFINICION DEL MODELO de RED

Nota: LA ultima Capa debe tener el numero de neuronas equivalente al total de materiales a distinguir.
Ojo si metemos la Cinta serían 4, Se podría obtener el numero de clases del BatchGenerator? SI, con n_clases obtenido del trainGenerator

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
# model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal'))
# model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(n_clases,activation='sigmoid'))

# summarize the model
model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [ ]:
# fit the model
# simple early stopping
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
es = EarlyStopping(monitor='loss', mode='auto', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='sparse_categorical_accuracy', mode='max', verbose=1, save_best_only=True)
# fit model
history= model.fit(traingenerator,epochs=100, batch_size=500, steps_per_epoch=10,  verbose=1, callbacks=[es, mc])
#history = model.fit(x,y, epochs=100, batch_size=500, steps_per_epoch=10,  verbose=1)

In [ ]:
# load the saved model
saved_model = load_model('best_model.h5')
# evaluate the model
#_, train_acc = saved_model.evaluate(trainX, trainy, verbose=0)
#_, test_acc = saved_model.evaluate(testX, testy, verbose=0)
#print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
# plot learning curves
plt.title('Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.plot(history.history['loss'], label='train')
#plt.plot(history.history['val_loss'], label='train')
plt.plot(history.history['sparse_categorical_accuracy'], label='acc')
plt.legend()
plt.show()

In [ ]:
filelist=testFiles
validationGenerator = ValidationGenerator(filelist, batchsize, ';')


In [ ]:
#loss, accuracy= model.evaluate(x,y)
loss, accuracy= saved_model.evaluate(validationGenerator)
print('Model Loss: %.2f, Accuracy: %.2f' % ((loss*100),(accuracy*100)))
#y_pred= model.predict(x,None);

#print(y_pred)

#loss, acc = model.evaluate(x,y, verbose=1, batch_size=100)
#print('Test Accuracy: %.3f' % acc)

### INFORME DE RESULTADOS OBTENIDOS - VALIDACION

Datos Nuevos capturados con la cinta negra - JOVISA_CSV_FOLDER
----------------------------------------
1- SIN TENER EN CUENTA LA CINTA NEGRA EN LOS DATOS DE ENTRADA

    a) Maxnorm : Loss: 5.18%, Accuracy: 99.83%

    b) NO Maxnorm: Loss: 4.45 %, Accuracy: 99.84%

    c) raw: Loss: 109.16%, Accuracy: 42.37%

2- INCLUYENDO LA CINTA NEGRA EN LOS DATOS DE ENTRADA

    a) Maxnorm : Loss: 9.92%, Accuracy: 96.61%

    b) NO Maxnorm: Loss: 16.71%, Accuracy: 92.23%

    c) raw: Loss: 247.41%, Accuracy: 87.11%

Datos caputados con la cinta AZUL -
----------------------------------------
1- SIN TENER EN CUENTA LA CINTA AZUL EN LOS DATOS DE ENTRADA

    a) Maxnorm : Loss: 1.69%, Accuracy: 99.62%

    b) NO Maxnorm: Loss:  4.54%, Accuracy: 98.96%

    c) raw: Loss: 543.15%, Accuracy: 96.76%
2- INCLUYENDO LA CINTA azul EN LOS DATOS DE ENTRADA

    a) Maxnorm : Loss: 26.4%, Accuracy: 91.96%

    b) NO Maxnorm: Loss:  45.77%, Accuracy:81.02%

    c) raw: Loss: 5212.28%, Accuracy: 5.56%

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
estimator = KerasClassifier(build_fn=model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, x, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import recall_score, cohen_kappa_score, precision_recall_fscore_support


#A=confusion_matrix(y, y_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=A)
#disp.plot()


#print(accuracy_score(Alltest_labels, Allpredict))
#print(recall_score(y, y_pred, average=None))
#print(cohen_kappa_score(y, y_pred))


### Sanity Check

Ejecutando la siguiente celda (varias veces) deberemos ver en colores diferentes distintos espectros.
Se obtienen en orden desde el BatchGenerator pero se podrían poner aleatorios

In [ ]:
#Sanity Check ################################################
# Cargamos un único batch.
for x, y in generator:
    print(x.shape, y.shape)
    numLamdas=x.shape[0]
    print(numLamdas)
    print(x,y)
    plt.figure
    for ix in range(numLamdas):
        #Ploteamos el spectrum si 1 - BOTELLA --> ROJO
        if y[ix]==0.:
            spectrum=x[ix]
            print("BOTELLA")
            plt.plot(spectrum, "r-")

    for ix in range(numLamdas):
        #Ploteamos el spectrum si 2 - PAPEL --> AZUL
        if y[ix]==1.:
            spectrum=x[ix]
            print("PAPEL")
            plt.plot(spectrum, "b-")

    for ix in range(numLamdas):
        #Ploteamos el spectrum si 3 - CINTA --> NEGRO
        if y[ix]==2.:
            spectrum=x[ix]
            print("CINTA")
            plt.plot(spectrum, "k-")

    for ix in range(numLamdas):
        #Ploteamos el spectrum si 2 - TAPON --> CYAN
        if y[ix]==3.:
            spectrum=x[ix]
            print("TAPON")
            plt.plot(spectrum, "c-")


    plt.show()
    break


## Definición del Modelo



Redefinimos el BatchGenerator para que extraiga de todo el train set

# # compile the model with other optimizer
# sgd = SGD(learning_rate=0.01, momentum=0.9)
sgd = SGD(learning_rate=0.001)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])